# Setup

In [3]:
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd

In [7]:
!pwd

/home/verdy/code/TomVerdy/twitter_kaggle_challenge


# Aperçu de la data

In [12]:
data = pd.read_csv("Data/train.csv")

In [10]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [15]:
data.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [21]:
data["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [36]:
data_test = pd.read_csv("Data/test.csv")

In [37]:
data_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


# On encode la data

In [41]:
X = data["text"]

In [42]:
y = data["target"]

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [44]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

In [45]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [55]:
# Embed the training sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)

In [56]:
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [48]:
X_train_pad.shape

(5100, 200, 100)

In [49]:
y_train.shape

(5100,)

# Premier modèle

In [50]:
word2vec.wv.vector_size

100

In [51]:
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.metrics import *

In [52]:
rnn = Sequential([
    layers.Masking(mask_value=0),
    layers.LSTM(20, activation="tanh"),
    layers.Dense(10, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

rnn.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

2023-01-11 10:15:26.736263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-11 10:15:26.744651: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-11 10:15:26.744740: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (verdy-System-Product-Name): /proc/driver/nvidia/version does not exist
2023-01-11 10:15:26.746047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [53]:
from tensorflow.keras import callbacks
es = callbacks.EarlyStopping(patience=3,restore_best_weights=True)

rnn.fit(X_train_pad, y_train, epochs=10, batch_size=16, verbose=1)


Epoch 1/10
319/319 [==============================] - 32s 82ms/step - loss: 0.6575 - accuracy: 0.6108
Epoch 2/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6493 - accuracy: 0.6212
Epoch 3/10
319/319 [==============================] - 26s 81ms/step - loss: 0.6454 - accuracy: 0.6265
Epoch 4/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6440 - accuracy: 0.6290
Epoch 5/10
319/319 [==============================] - 29s 90ms/step - loss: 0.6392 - accuracy: 0.6351
Epoch 6/10
319/319 [==============================] - 28s 88ms/step - loss: 0.6379 - accuracy: 0.6380
Epoch 7/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6338 - accuracy: 0.6390
Epoch 8/10
319/319 [==============================] - 29s 90ms/step - loss: 0.6279 - accuracy: 0.6433
Epoch 9/10
319/319 [==============================] - 28s 89ms/step - loss: 0.6247 - accuracy: 0.6447
Epoch 10/10
319/319 [==============================] - 28s 88ms/step - loss: 0.619

In [57]:
rnn.evaluate(X_test_pad, y_test)

79/79 [==============================] - 3s 25ms/step - loss: 0.6336 - accuracy: 0.6327


[0.6336041688919067, 0.6327099204063416]

# Modele si on clean les données ?

## data cleaning

In [58]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [59]:
test

NameError: name 'test' is not defined